# Import library

In [2]:
import os
import os, sys
import glob
import re
import datetime
from datetime import datetime, timedelta 
import math, copy, time
from itertools import cycle
from collections import defaultdict

from parse import parse
import numpy as np
import pandas as pd

import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

# import torchvision
# import torchvision.transforms as transforms

from sklearn.metrics import roc_auc_score, plot_roc_curve
from sklearn import svm


from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn

%load_ext autoreload
%autoreload 2

In [3]:
pd.set_option("display.max_column", 100)

In [4]:
# Util functions
def size_and_ratio(df, col, dropna=True):
    if pd.__version__ > '1.1':
        sr = df.groupby(col, dropna=dropna).size().sort_values(ascending=False)
    elif dropna:
        sr = df.groupby(col).size().sort_values(ascending=False)
    else:  # dropna == False
        sr = df.fillna('nan').groupby(col).size().sort_values(ascending=False)
    sr_ratio = sr.copy() / sum(sr)
    print("Sum :", sum(sr), sr.shape)
    sr.name = 'size'
    sr_ratio.name = 'ratio'
    return pd.concat([sr, sr_ratio], axis=1)

# Data load

In [5]:
df_train = pd.read_csv("data/df_data_v3_1/df_train_v3_1.csv")
df_val = pd.read_csv("data/df_data_v3_1/df_valid_v3_1.csv")
df_test = pd.read_csv("data/df_data_v3_1/df_test_v3_1.csv")
print(df_train.shape, df_val.shape, df_test.shape)
df_train.head()

(14093, 24) (4647, 24) (4586, 24)


,baby_id,cough,fever,sore_throat,shortness_of_breath,head_ache,runny_nose,muscle_pain,chills,loss_of_taste,loss_of_smell,sputum,chest_pain,indication_other,indication_abroad,indication_contact,global_confirmed_ratio,confirmed_ratio,sigungu_confirmed_ratio,mask,gender,age_ratio,weekday,pcr_result
0,21099,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0.173311,0.007956,0.012987,1,0.0,0.18,3,0
1,21151,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0.179009,0.008614,0.013084,1,0.0,0.17,4,0
2,21024,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0.179009,0.008614,0.013084,1,1.0,0.29,4,0
3,21176,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0.131500,0.008252,0.000000,0,0.0,0.40,5,0
4,4467,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0.131500,0.008252,0.000000,0,1.0,0.34,5,0


In [6]:
import random
print("Train:", df_train.shape, df_train['baby_id'].nunique())
print("Validaiton:", df_val.shape, df_val['baby_id'].nunique())

Train: (14093, 24) 9641
Validaiton: (4647, 24) 3213


## Remove baby_id and results

In [7]:
exclude_cols = ['baby_id', 'pcr_result']
cols = [x for x in df_train.columns if x not in exclude_cols]

df_train_gt = df_train[['pcr_result']]
df_train_input = df_train[cols]
df_val_gt = df_val[['pcr_result']]
df_val_input = df_val[cols]
df_test_gt = df_test[['pcr_result']]
df_test_input = df_test[cols]


# Make category values

In [8]:
from model.attention_model import CategoricalAttentionModel
from model.utils import get_dict_category_from_dataset
from model.utils import get_category_key, create_cat_and_intensity_from_df

In [9]:
df_trainval = pd.concat([df_train, df_val], axis=0)
print(df_trainval.shape)

dict_category, df_trainval_typed = get_dict_category_from_dataset(df_trainval[cols])
_, df_train_typed = get_dict_category_from_dataset(df_train[cols])
_, df_val_typed = get_dict_category_from_dataset(df_val[cols])
_, df_test_typed = get_dict_category_from_dataset(df_test[cols])
dict_category

(18740, 24)
cough category [1 0] ...
fever category [0 1] ...
sore_throat category [0 1] ...
shortness_of_breath category [0 1] ...
head_ache category [1 0] ...
runny_nose category [0 1] ...
muscle_pain category [0 1] ...
chills category [0 1] ...
loss_of_taste category [0 1] ...
loss_of_smell category [0 1] ...
sputum category [1 0] ...
chest_pain category [0 1] ...
indication_other category [0 1] ...
indication_abroad category [0 1] ...
indication_contact category [1 0] ...
global_confirmed_ratio float [0.1733114  0.17900909 0.13149977 0.14631498 0.15655098 0.17425644
 0.17103502 0.16990724 0.12749546 0.14885249]
confirmed_ratio float [0.00795568 0.00861396 0.00825183 0.00797017 0.01154644 0.01142734
 0.0113018  0.01122937 0.01076423 0.00895517]
sigungu_confirmed_ratio float [0.01298674 0.01308374 0.         0.01324623 0.01342084 0.0135712
 0.01798011 0.01397984 0.01779546 0.0141302 ]
mask category [1 0] ...
gender category [0. 1.] ...
age_ratio float [0.18 0.17 0.29 0.4  0.34 0.41 0

{'cough_1.0': 0,
 'cough_0.0': 1,
 'fever_0.0': 2,
 'fever_1.0': 3,
 'sore_throat_0.0': 4,
 'sore_throat_1.0': 5,
 'shortness_of_breath_0.0': 6,
 'shortness_of_breath_1.0': 7,
 'head_ache_1.0': 8,
 'head_ache_0.0': 9,
 'runny_nose_0.0': 10,
 'runny_nose_1.0': 11,
 'muscle_pain_0.0': 12,
 'muscle_pain_1.0': 13,
 'chills_0.0': 14,
 'chills_1.0': 15,
 'loss_of_taste_0.0': 16,
 'loss_of_taste_1.0': 17,
 'loss_of_smell_0.0': 18,
 'loss_of_smell_1.0': 19,
 'sputum_1.0': 20,
 'sputum_0.0': 21,
 'chest_pain_0.0': 22,
 'chest_pain_1.0': 23,
 'indication_other_0.0': 24,
 'indication_other_1.0': 25,
 'indication_abroad_0.0': 26,
 'indication_abroad_1.0': 27,
 'indication_contact_1.0': 28,
 'indication_contact_0.0': 29,
 'global_confirmed_ratio': 30,
 'confirmed_ratio': 31,
 'sigungu_confirmed_ratio': 32,
 'mask_1.0': 33,
 'mask_0.0': 34,
 'gender_0.0': 35,
 'gender_1.0': 36,
 'age_ratio': 37,
 'weekday_3.0': 38,
 'weekday_4.0': 39,
 'weekday_5.0': 40,
 'weekday_0.0': 41,
 'weekday_1.0': 42,
 'wee

### category 값, intensity 값 생성

### train 데이터 먼저

In [ ]:
df_train_category, df_train_intensity = create_cat_and_intensity_from_df(df_train_typed, dict_category)

display(df_train_category.sample(10))
display(df_train_intensity.sample(10))

예시로 sore_throat id값 확인해봄

In [ ]:
size_and_ratio(df_train_category, 'sore_throat')

### df_val에서 df_cat값과 df_intensity 값 생성

In [ ]:
df_val_category, df_val_intensity = create_cat_and_intensity_from_df(df_val_typed, dict_category)


### df_test에서도 마찬가지로 생성

In [ ]:
df_test_category, df_test_intensity = create_cat_and_intensity_from_df(df_test_typed, dict_category)


In [ ]:
train_np_cat = df_train_category.values
train_np_ins = df_train_intensity.values
train_np_gt = df_train_gt[['pcr_result']].values

val_np_cat = df_val_category.values
val_np_ins = df_val_intensity.values
val_np_gt = df_val_gt[['pcr_result']].values

test_np_cat = df_test_category.values
test_np_ins = df_test_intensity.values
test_np_gt = df_test_gt[['pcr_result']].values

train_np_gt, val_np_gt, test_np_gt

In [ ]:
train_input = [train_np_cat.copy(), train_np_ins.copy()]
train_gt = train_np_gt.copy()

valid_input = [val_np_cat.copy(), val_np_ins.copy()]
valid_gt = val_np_gt.copy()

test_input = [test_np_cat.copy(), test_np_ins.copy()]
test_gt = test_np_gt.copy()


In [ ]:
print(train_input[0].shape, valid_input[0].shape, test_input[0].shape)
print(train_gt.shape, valid_gt.shape, test_gt.shape)

In [ ]:
print("# of train data positive: ", train_gt.sum())
print("# of train data negative: ", train_gt.shape[0] - train_gt.sum())
print("ratio of P/N: ", train_gt.sum()/(train_gt.shape[0] - train_gt.sum()))
print("")

print("# of valid data positive: ", valid_gt.sum())
print("# of valid data negative: ", valid_gt.shape[0] - valid_gt.sum())
print("ratio of P/N: ", valid_gt.sum()/(valid_gt.shape[0] - valid_gt.sum()))
print("")

print("# of test data positive: ", test_gt.sum())
print("# of test data negative: ", test_gt.shape[0] - test_gt.sum())
print("ratio of P/N: ", test_gt.sum()/(test_gt.shape[0] - test_gt.sum()))

### Check dict_category

In [ ]:
dict_category

## Let's Train imported class (simple attention module without inesity encoding)

In [ ]:
from model.attention_model import CategoricalAtentionMultiplyIntensityModel, CategoricalAttentionAddIntensityModel
from model.simple_attention import CategoricalAttentionSimpleModel
from model.base_dataloader import IsraelDataLoader, IsraelDatasetWithIntensity
from model.utils import run_validation

In [ ]:
train_input

In [ ]:
print("cat value:", train_input[0].shape)
print("intensity value:", train_input[1].shape)
print("Ground truth value:", train_gt.shape)

# Dataset, data_loader

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# creat dataset

train_dataset = IsraelDatasetWithIntensity(
    train_np_cat,
    train_np_ins,
    train_np_gt,
    dict_category
)

val_dataset = IsraelDatasetWithIntensity(
    val_np_cat,
    val_np_ins,
    val_np_gt,
    dict_category
)

test_dataset = IsraelDatasetWithIntensity(
    test_np_cat, 
    test_np_ins,
    test_np_gt,
    dict_category
)

## CategoricalAttentionSimpleModel

In [91]:
d_model = 128
n_head = 2

model = CategoricalAttentionSimpleModel(
    dict_category, d_model=d_model, n_head=n_head, len_seq=train_dataset.dataset_np_cat.shape[1]
)

print("d_model:", model.d_model, "n_head:", model.n_head, 'len_seq:', model.len_seq)
loss_func = nn.BCELoss()
optimizer = optim.RAdam(model.parameters(), lr=3*1e-5)
batch_size = 64
epochs = 10

train_loader = IsraelDataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = IsraelDataLoader(val_dataset, batch_size=batch_size, shuffle=False)

train_log = dict()
val_log = dict()
loss_func = nn.BCELoss()
val_metrics = {
    "loss": loss_func,
    'auc': lambda pred, gt: roc_auc_score(gt.numpy(), pred.numpy())
}

import copy

model.train()
for epoch in range(epochs):
    # **************** Train ****************
    train_result_info = CategoricalAttentionSimpleModel.train_on_epoch(epoch, train_loader, model, loss_func, optimizer)
    train_log[f'epoch_train_{epoch}'] = train_result_info
    logger.debug(f"epoch {epoch} train finished. train_result_info:{train_result_info}")
    
    # ************** validation *********************
    model.eval()
    val_ts_cat = torch.LongTensor(val_np_cat)
    val_ts_gt = torch.FloatTensor(val_np_gt)
    val_result = run_validation(model, val_metrics, val_ts_cat, val_ts_gt)
    
    val_log[f'epoch_val_{epoch}'] = val_result

    logger.info(f"epoch {epoch} val auc: {val_result['auc']}")

    if not val_log:
        best_model = copy.deepcopy(model)
    elif val_result['auc'] >= max(val_log.values(), key=lambda x : x['auc'])['auc']:
        best_model = copy.deepcopy(model)
    

d_model: 128 n_head: 2 len_seq: 22


INFO:model.simple_attention:Epoch Step: 0 Train Loss: 0.028435930609703064 elapsed: 2.479694128036499
INFO:__main__:epoch 0 val auc: 0.7865921688121542
INFO:model.simple_attention:Epoch Step: 1 Train Loss: 0.025547616183757782 elapsed: 1.4231297969818115
INFO:__main__:epoch 1 val auc: 0.7860069926413265
INFO:model.simple_attention:Epoch Step: 2 Train Loss: 0.022680940106511116 elapsed: 1.4236347675323486
INFO:__main__:epoch 2 val auc: 0.7866531345667509
INFO:model.simple_attention:Epoch Step: 3 Train Loss: 0.023071130737662315 elapsed: 1.4312961101531982
INFO:__main__:epoch 3 val auc: 0.7899979321556921
INFO:model.simple_attention:Epoch Step: 4 Train Loss: 0.02349257655441761 elapsed: 1.4623210430145264
INFO:__main__:epoch 4 val auc: 0.793574589758704
INFO:model.simple_attention:Epoch Step: 5 Train Loss: 0.021567175164818764 elapsed: 1.4093742370605469
INFO:__main__:epoch 5 val auc: 0.7932440911943109
INFO:model.simple_attention:Epoch Step: 6 Train Loss: 0.021651579067111015 elapsed: 1

In [94]:
## Check the best model's validation performance
display(val_log)
max(val_log.values(), key=lambda x : x['auc'])


{'epoch_val_0': {'loss': tensor(0.6024), 'auc': 0.7865921688121542},
 'epoch_val_1': {'loss': tensor(0.5191), 'auc': 0.7860069926413265},
 'epoch_val_2': {'loss': tensor(0.5376), 'auc': 0.7866531345667509},
 'epoch_val_3': {'loss': tensor(0.4726), 'auc': 0.7899979321556921},
 'epoch_val_4': {'loss': tensor(0.4921), 'auc': 0.793574589758704},
 'epoch_val_5': {'loss': tensor(0.4976), 'auc': 0.7932440911943109},
 'epoch_val_6': {'loss': tensor(0.4863), 'auc': 0.7890143988515146},
 'epoch_val_7': {'loss': tensor(0.5070), 'auc': 0.7887827765206974},
 'epoch_val_8': {'loss': tensor(0.5387), 'auc': 0.792437394230952},
 'epoch_val_9': {'loss': tensor(0.4729), 'auc': 0.7913313352094464}}

{'loss': tensor(0.4921), 'auc': 0.793574589758704}


## CategoricalAtention MultiplyIntensity Model

In [95]:
d_model = 128
n_head = 2

model = CategoricalAtentionMultiplyIntensityModel(
    dict_category, d_model=d_model, n_head=n_head, len_seq=train_dataset.dataset_np_cat.shape[1],
)

print("d_model:", model.d_model, "n_head:", model.n_head, 'len_seq:', model.len_seq)
loss_func = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=3*1e-5)
batch_size = 32
epochs = 10

train_loader = IsraelDataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = IsraelDataLoader(val_dataset, batch_size=batch_size, shuffle=False)

train_log = dict()
val_log = dict()
loss_func = nn.BCELoss()
val_metrics = {
    "loss": loss_func,
    'auc': lambda pred, gt: roc_auc_score(gt.numpy(), pred.numpy())
}

model.train()
for epoch in range(epochs):
    # **************** Train ****************
    train_result_info = CategoricalAtentionMultiplyIntensityModel.train_on_epoch(epoch, train_loader, model, loss_func, optimizer)
    train_log[f'epoch_train_{epoch}'] = train_result_info
    logger.debug(f"epoch {epoch} train finished. train_result_info:{train_result_info}")
    
    # ************** validation *********************
    val_result = CategoricalAtentionMultiplyIntensityModel.validation_on_epoch(epoch, model, val_loader, val_metrics)
    val_log[f'epoch_val_{epoch}'] = val_result

    logger.info(f"epoch {epoch} val auc: {val_result['auc']}")

    if not val_log:
        best_model = model.copy()
    elif val_result['auc'] >= max(val_log.values(), key=lambda x : x['auc'])['auc']:
        best_model = model.copy()

d_model: 128 n_head: 2 len_seq: 22


INFO:model.base_model:Epoch Step: 0 Train Loss: 0.017019765451550484 elapsed: 1.7608428001403809
INFO:model.base_model:cur_epoch:0, validation elapsed_time:0.2615170478820801
INFO:__main__:epoch 0 val auc: 0.8003089882299252
INFO:model.base_model:Epoch Step: 1 Train Loss: 0.01570836827158928 elapsed: 2.9120898246765137
INFO:model.base_model:cur_epoch:1, validation elapsed_time:0.18551397323608398
INFO:__main__:epoch 1 val auc: 0.7909983409708886
INFO:model.base_model:Epoch Step: 2 Train Loss: 0.015375824645161629 elapsed: 2.8725366592407227
INFO:model.base_model:cur_epoch:2, validation elapsed_time:0.19037389755249023
INFO:__main__:epoch 2 val auc: 0.79646957178985
INFO:model.base_model:Epoch Step: 3 Train Loss: 0.016488058492541313 elapsed: 3.175119161605835
INFO:model.base_model:cur_epoch:3, validation elapsed_time:0.18777108192443848
INFO:__main__:epoch 3 val auc: 0.8143900572341276
INFO:model.base_model:Epoch Step: 4 Train Loss: 0.0175065528601408 elapsed: 3.119485855102539
INFO:mo

In [96]:
## Check the best model's validation performance
display(val_log)
max(val_log.values(), key=lambda x : x['auc'])

## Check the best model's test performance
test_loader = IsraelDataLoader(test_dataset, batch_size=64, shuffle=False)

best_model.eval()
test_result = model.validation_on_epoch(epoch, best_model, test_loader, val_metrics)
test_result

{'epoch_val_0': {'loss': tensor(0.4966), 'auc': 0.8003089882299252},
 'epoch_val_1': {'loss': tensor(0.5366), 'auc': 0.7909983409708886},
 'epoch_val_2': {'loss': tensor(0.5421), 'auc': 0.79646957178985},
 'epoch_val_3': {'loss': tensor(0.6148), 'auc': 0.8143900572341276},
 'epoch_val_4': {'loss': tensor(0.6852), 'auc': 0.7659887147991102},
 'epoch_val_5': {'loss': tensor(0.5384), 'auc': 0.7502872402122036},
 'epoch_val_6': {'loss': tensor(0.6561), 'auc': 0.521425719229526},
 'epoch_val_7': {'loss': tensor(0.5823), 'auc': 0.6534248968454679},
 'epoch_val_8': {'loss': tensor(0.9448), 'auc': 0.7120475699264133},
 'epoch_val_9': {'loss': tensor(0.6347), 'auc': 0.794830745754977}}

INFO:model.base_model:cur_epoch:9, validation elapsed_time:0.16588401794433594


{'loss': tensor(0.6643), 'auc': 0.8135130846597012}


## CategoricalAtention MultiplyIntensity Model with dropout

In [97]:
d_model = 128
n_head = 2

model = CategoricalAtentionMultiplyIntensityModel(
    dict_category, d_model=d_model, n_head=n_head, len_seq=train_dataset.dataset_np_cat.shape[1],
    dropout=0.3
)

print("d_model:", model.d_model, "n_head:", model.n_head, 'len_seq:', model.len_seq)
loss_func = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=3*1e-5)
batch_size = 32
epochs = 10

train_log = dict()
val_log = dict()
loss_func = nn.BCELoss()
val_metrics = {
    "loss": loss_func,
    'auc': lambda pred, gt: roc_auc_score(gt.numpy(), pred.numpy())
}

model.train()
for epoch in range(epochs):
    # **************** Train ****************
    train_result_info = CategoricalAtentionMultiplyIntensityModel.train_on_epoch(epoch, train_loader, model, loss_func, optimizer)
    train_log[f'epoch_train_{epoch}'] = train_result_info
    logger.debug(f"epoch {epoch} train finished. train_result_info:{train_result_info}")
    
    # ************** validation *********************
    val_result = CategoricalAtentionMultiplyIntensityModel.validation_on_epoch(epoch, model, val_loader, val_metrics)
    val_log[f'epoch_val_{epoch}'] = val_result

    logger.info(f"epoch {epoch} val auc: {val_result['auc']}")

    if not val_log:
        best_model = model.copy()
    elif val_result['auc'] >= max(val_log.values(), key=lambda x : x['auc'])['auc']:
        best_model = model.copy()
    

d_model: 128 n_head: 2 len_seq: 22


INFO:model.base_model:Epoch Step: 0 Train Loss: 0.017478717491030693 elapsed: 3.693838119506836
INFO:model.base_model:cur_epoch:0, validation elapsed_time:0.18227720260620117
INFO:__main__:epoch 0 val auc: 0.8002827242303817
INFO:model.base_model:Epoch Step: 1 Train Loss: 0.015553481876850128 elapsed: 3.01898193359375
INFO:model.base_model:cur_epoch:1, validation elapsed_time:0.1799449920654297
INFO:__main__:epoch 1 val auc: 0.7999102745716948
INFO:model.base_model:Epoch Step: 2 Train Loss: 0.017034394666552544 elapsed: 3.083362102508545
INFO:model.base_model:cur_epoch:2, validation elapsed_time:0.18827533721923828
INFO:__main__:epoch 2 val auc: 0.8141896902511837
INFO:model.base_model:Epoch Step: 3 Train Loss: 0.015669047832489014 elapsed: 3.1451220512390137
INFO:model.base_model:cur_epoch:3, validation elapsed_time:0.18166017532348633
INFO:__main__:epoch 3 val auc: 0.7927374693388602
INFO:model.base_model:Epoch Step: 4 Train Loss: 0.016531746834516525 elapsed: 3.1298530101776123
INFO

In [98]:
## Check the best model's validation performance
display(val_log)
print("Best validation performance:", max(val_log.values(), key=lambda x : x['auc']))

## Check the best model's test performance
test_loader = IsraelDataLoader(test_dataset, batch_size=64, shuffle=False)

best_model.eval()
test_result = best_model.validation_on_epoch(epoch, best_model, test_loader, val_metrics)
test_result

{'epoch_val_0': {'loss': tensor(0.4756), 'auc': 0.8002827242303817},
 'epoch_val_1': {'loss': tensor(0.4703), 'auc': 0.7999102745716948},
 'epoch_val_2': {'loss': tensor(0.5246), 'auc': 0.8141896902511837},
 'epoch_val_3': {'loss': tensor(0.5563), 'auc': 0.7927374693388602},
 'epoch_val_4': {'loss': tensor(0.5064), 'auc': 0.8004738215664278},
 'epoch_val_5': {'loss': tensor(0.5159), 'auc': 0.8030500703542431},
 'epoch_val_6': {'loss': tensor(0.7625), 'auc': 0.7925388849803199},
 'epoch_val_7': {'loss': tensor(0.5992), 'auc': 0.7926063870243958},
 'epoch_val_8': {'loss': tensor(0.6542), 'auc': 0.7519579158030842},
 'epoch_val_9': {'loss': tensor(0.4765), 'auc': 0.795687950409766}}

INFO:model.base_model:cur_epoch:9, validation elapsed_time:0.1581571102142334


Best validation performance: {'loss': tensor(0.5246), 'auc': 0.8141896902511837}


{'loss': tensor(0.5175), 'auc': 0.8159574834718576}

## Transformer 모델 
-> 쓸데없이 decoder레이어 있어서 학습 잘 안되는것 같아 빼버림

## Transformer encoder
일단 3층짜리로 한번 돌려보자

In [62]:
from model.attention_model import CategoricalTransformerEncoderMultiplyIntensity, CategoricalTransformerEncoderAddIntensity

In [1]:
batch_size = 32
epochs = 10

train_loader = IsraelDataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = IsraelDataLoader(val_dataset, batch_size=batch_size, shuffle=False)

len_seq = train_input[0].shape[1]
print("len_seq:", len_seq)
d_model = 64
n_head = 4
loss_func = nn.BCELoss()

encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_head)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=3)
model = CategoricalTransformerEncoderMultiplyIntensity(
    transformer_encoder=transformer_encoder,
    dict_category=dict_category,
    len_seq=len_seq,
    dropout=0.3
)

model.eval()

print("d_model:", model.d_model, "n_head:", model.n_head, 'len_seq:', model.len_seq)
loss_func = nn.BCELoss()
optimizer = optim.RAdam(model.parameters(), lr=1e-4)

train_log = dict()
val_log = dict()
loss_func = nn.BCELoss()
val_metrics = {
    "loss": loss_func,
    'auc': lambda pred, gt: roc_auc_score(gt.numpy(), pred.numpy())
}


for epoch in range(epochs):
    # **************** Train ****************
    train_result_info = CategoricalTransformerEncoderMultiplyIntensity.train_on_epoch(epoch, train_loader, model, loss_func, optimizer, verbose=True)
    train_log[f'epoch_train_{epoch}'] = train_result_info
    logger.debug(f"epoch {epoch} train finished. train_result_info:{train_result_info}")
    
    # ************** validation *********************
    val_result = CategoricalTransformerEncoderMultiplyIntensity.validation_on_epoch(epoch, model, val_loader, val_metrics)
    val_log[f'epoch_val_{epoch}'] = val_result
    logger.info(f"epoch {epoch} val result: {val_result}")

    if not val_log:
        best_model = model.copy()
    elif val_result['auc'] >= max(val_log.values(), key=lambda x : x['auc'])['auc']:
        best_model = model.copy()
    

NameError: name 'IsraelDataLoader' is not defined

In [64]:
val_log

{'epoch_val_0': {'loss': tensor(0.6341), 'auc': 0.8024701232149989},
 'epoch_val_1': {'loss': tensor(0.5979), 'auc': 0.8056727862181743},
 'epoch_val_2': {'loss': tensor(0.4554), 'auc': 0.8045673214048031},
 'epoch_val_3': {'loss': tensor(0.5578), 'auc': 0.8065964233423969},
 'epoch_val_4': {'loss': tensor(0.5047), 'auc': 0.8079625078435473},
 'epoch_val_5': {'loss': tensor(0.5876), 'auc': 0.8005703209674659},
 'epoch_val_6': {'loss': tensor(0.5164), 'auc': 0.8000219857009754},
 'epoch_val_7': {'loss': tensor(0.4785), 'auc': 0.7945602622121655},
 'epoch_val_8': {'loss': tensor(0.4757), 'auc': 0.7988025517674127},
 'epoch_val_9': {'loss': tensor(0.6145), 'auc': 0.8062426318191325}}

In [65]:
max(val_log.values(), key=lambda x : x['auc'])

{'loss': tensor(0.5047), 'auc': 0.8079625078435473}

In [69]:

test_loader = IsraelDataLoader(test_dataset, batch_size=64, shuffle=False)

best_model.eval()
test_result = CategoricalTransformerEncoderMultiplyIntensity.validation_on_epoch(epoch, best_model, test_loader, val_metrics)
test_result

INFO:model.base_model:cur_epoch:0, validation elapsed_time:1.0178778171539307


{'loss': tensor(0.5296), 'auc': 0.8023155434422573}